In [2]:
import fastrepl
from fastrepl.llama_index import FastREPLCallbackHandler

ds = fastrepl.Dataset.from_dict({})

evaluator = fastrepl.RAGEvaluator(
    node=fastrepl.RAGAS(model="gpt-3.5-turbo", metric="Faithfulness")
)

fastrepl_callback_handler = FastREPLCallbackHandler(dataset=ds)

In [3]:
from llama_index import ServiceContext
from llama_index.callbacks import CallbackManager

callback_manager = CallbackManager([fastrepl_callback_handler])
service_context = ServiceContext.from_defaults(callback_manager=callback_manager)

In [4]:
from llama_index import SimpleWebPageReader

urls = ["http://paulgraham.com/greatwork.html"]
documents = SimpleWebPageReader(html_to_text=True).load_data(urls)
assert len(documents) == 1

In [5]:
from llama_index import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine(service_context=service_context)

In [6]:
questions = [
    "hi!",
    "How does the author suggest that one can determine if a field is suited for them?",
    "How does the author recommend ensuring that something created for others is something they actually want?",
]

for q in questions:
    query_engine.query(q)

# TODO: This is not working. Thread Safety?
# _ = fastrepl.local_runner(fn=query_engine.query).run(
#     args_list=[(q,) for q in questions]
# )

In [7]:
ds

fastrepl.Dataset({
    features: ['question', 'answer', 'contexts'],
    num_rows: 3
})

In [8]:
result = fastrepl.local_runner(evaluator=evaluator, dataset=ds).run()
result

Output()

fastrepl.Dataset({
    features: ['question', 'answer', 'contexts', 'result'],
    num_rows: 3
})

In [9]:
result["result"]

[0.0, 1.0, 1.0]